In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["OMP_NUM_THREADS"] = "12"

import torch

In [ ]:
import time
import yaml
import pandas as pd
from rdkit import Chem

Please note the code here run on Docker instead of Google Colab. So it has to be adequately rewritten to run on Colab. However, setting the environment on Google Colab is in RASSP_predict notebook.

In [ ]:
from rassp.msutil.masscompute import FragmentFormulaPeakEnumerator

#valid_atoms = {1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53}
valid_atoms = {1, 6, 7, 8, 9, 15, 16, 17}
num_peaks_per_formula = 12
#max_formulae = 100000
max_formulae = 4096

ffe = FragmentFormulaPeakEnumerator(sorted(valid_atoms), use_highres=True, max_peak_num=num_peaks_per_formula)

def goodmol(mol):
        if len(mol.GetAtoms()) > 48:
            return False
        
        atoms = { a.GetAtomicNum() for a in mol.GetAtoms() }
        if not atoms < valid_atoms:
            return False
            
        f,m = ffe.get_frag_formulae(mol)
        if len(f) > max_formulae:
            return False

        return True

In [ ]:
test = pd.read_parquet('test.pq')
test

In [ ]:
smalltest = test[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in test.rdmol ]]
smalltest

In [ ]:
smalltest.to_parquet('smalltest.pq')

In [ ]:
train = pd.read_parquet('train.pq')
train

In [ ]:
smalltrain = train[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in train.rdmol ]]
smalltrain

In [ ]:
smalltrain.to_parquet('smalltrain.pq')